# Cricket Player Embeddings
## Thomas Body

In [138]:
from pathlib import Path
import pandas as pd
import numpy as np
from fastai.tabular import *
from fastai import *
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

In [2]:
# Path file to the data
path = Path("data")
files = list(path.iterdir())
files

[PosixPath('data/bowlers_combined_data.csv'),
 PosixPath('data/batters_combined_data.csv')]

## Creating, inspecting, and cleaning the batters and bowlers dataframes

In [3]:
# Creates the batters and bowlers dataframes from the csv files
batters_df = pd.read_csv(files[1])
bowlers_df = pd.read_csv(files[0])

# Creating a list of both dataframes
dfs = [batters_df, bowlers_df]

In [4]:
bowlers_df.columns

Index(['Unnamed: 0', 'Player', 'Overs', 'Maidens', 'Runs', 'Wickets',
       'EconRate', 'Date', 'Location', 'Result', 'Innings', 'Team',
       'BatSideRR', 'BatSideWicketsLost', 'BatSideScore', 'Win'],
      dtype='object')

In [5]:
batters_df.columns

Index(['Unnamed: 0', 'Player', 'Dismissal', 'Runs', 'BallsFaced', 'Fours',
       'Sixes', 'StrikeRate', 'Date', 'Location', 'Result', 'Innings', 'Team',
       'BatSideRR', 'BatSideWicksLost', 'BatSideScore', 'Win'],
      dtype='object')

In [6]:
# Dropping the Unnamed: 0 column from both dataframes
for df in dfs:
    df.drop("Unnamed: 0", axis=1, inplace=True)

In [7]:
batters_df.head().T

0  \
Player                                       Mudassar Nazar   
Dismissal                                       b  De Silva   
Runs                                                     27   
BallsFaced                                               57   
Fours                                                     1   
Sixes                                                     0   
StrikeRate                                            47.37   
Date                                       29th March, 1982   
Location                            Gaddafi Stadium, Lahore   
Result            Sri Lanka won by 30 runs [Revised Target]   
Innings                                                   1   
Team                                               Pakistan   
BatSideRR                                              5.98   
BatSideWicksLost                                          4   
BatSideScore                                            239   
Win                                                       0   

                                                          1  \
Player                                          Mohsin Khan   
Dismissal                                           run out   
Runs                                                      6   
BallsFaced                                               14   
Fours                                                     0   
Sixes                                                     0   
StrikeRate                                            42.86   
Date                                       29th March, 1982   
Location                            Gaddafi Stadium, Lahore   
Result            Sri Lanka won by 30 runs [Revised Target]   
Innings                                                   1   
Team                                               Pakistan   
BatSideRR                                              5.98   
BatSideWicksLost                                          4   
BatSideScore                                            239   
Win                                                       0   

                                                          2  \
Player                                         Zaheer Abbas   
Dismissal                         c  Madugalle b  Ratnayeke   
Runs                                                    123   
BallsFaced                                               87   
Fours                                                    15   
Sixes                                                     3   
StrikeRate                                           141.38   
Date                                       29th March, 1982   
Location                            Gaddafi Stadium, Lahore   
Result            Sri Lanka won by 30 runs [Revised Target]   
Innings                                                   1   
Team                                               Pakistan   
BatSideRR                                              5.98   
BatSideWicksLost                                          4   
BatSideScore                                            239   
Win                                                       0   

                                                          3  \
Player                                        Javed Miandad   
Dismissal                                           run out   
Runs                                                      1   
BallsFaced                                                7   
Fours                                                     0   
Sixes                                                     0   
StrikeRate                                            14.29   
Date                                       29th March, 1982   
Location                            Gaddafi Stadium, Lahore   
Result            Sri Lanka won by 30 runs [Revised Target]   
Innings                                                   1   
Team                                               Pakistan   
BatSideRR              

In [8]:
bowlers_df.head().T

0  \
Player                                           A L F De Mel   
Overs                                                       8   
Maidens                                                     1   
Runs                                                       31   
Wickets                                                     0   
EconRate                                                 3.88   
Date                                         29th March, 1982   
Location                              Gaddafi Stadium, Lahore   
Result              Sri Lanka won by 30 runs [Revised Target]   
Innings                                                     1   
Team                                                Sri Lanka   
BatSideRR                                                5.98   
BatSideWicketsLost                                          4   
BatSideScore                                              239   
Win                                                         1   

                                                            1  \
Player                                          J R Ratnayeke   
Overs                                                       8   
Maidens                                                     1   
Runs                                                       42   
Wickets                                                     1   
EconRate                                                 5.25   
Date                                         29th March, 1982   
Location                              Gaddafi Stadium, Lahore   
Result              Sri Lanka won by 30 runs [Revised Target]   
Innings                                                     1   
Team                                                Sri Lanka   
BatSideRR                                                5.98   
BatSideWicketsLost                                          4   
BatSideScore                                              239   
Win                                                         1   

                                                            2  \
Player                                         A N Ranasinghe   
Overs                                                       6   
Maidens                                                     0   
Runs                                                       33   
Wickets                                                     0   
EconRate                                                  5.5   
Date                                         29th March, 1982   
Location                              Gaddafi Stadium, Lahore   
Result              Sri Lanka won by 30 runs [Revised Target]   
Innings                                                     1   
Team                                                Sri Lanka   
BatSideRR                                                5.98   
BatSideWicketsLost                                          4   
BatSideScore                                              239   
Win                                                         1   

                                                            3  \
Player                                            B Warnapura   
Overs                                                       2   
Maidens                                                     0   
Runs                                                       21   
Wickets                                                     0   
EconRate                                                 10.5   
Date                                         29th March, 1982   
Location                              Gaddafi Stadium, Lahore   
Result              Sri Lanka won by 30 runs [Revised Target]   
Innings                                                     1   
Team                                                Sri Lanka   
BatSideRR                                                5.98   
BatSideWicketsLost                                          4   
BatSideScore               

In [9]:
# Converting dates to datetime
for df in dfs:
    df['Date'] = pd.to_datetime(df['Date'])

In [10]:
# Add datepart function
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [11]:
# Adding dateparts
for df in dfs:
    add_datepart(df, 'Date', drop=False)
    df.drop('Elapsed', axis=1, inplace=True)

In [12]:
# Splitting out the winning team and the result
for df in dfs:
    df['WinningTeam'] = df['Result'].str.split(' won').str[0]
    df['WonByHowMuch'] = df['Result'].str.split(' won').str[1].str.split(' by').str[1].str.split().str[0]
    df['WinByHowType'] = df['Result'].str.split(' won').str[1].str.split(' by').str[1].str.split().str[1]

In [13]:
# Replacing NaN values in batters_df with DNB (Did not bat)
for col in ['Dismissal', 'Runs', 'BallsFaced', 'Fours', 'Sixes', 'StrikeRate']:
    batters_df.loc[:, col].fillna('DNB', inplace=True)

In [14]:
# Removing tied matches and no results for simplicity
bowlers_df = bowlers_df[bowlers_df['Result']!=('No result' and 'Match tied')]
batters_df = batters_df[batters_df['Result']!=('No result' and 'Match tied')]

In [15]:
# Removing conceded matches
# Bowlers
bowlers_df = bowlers_df[bowlers_df.WonByHowMuch.isna()==False]
bowlers_df = bowlers_df[bowlers_df.WonByHowMuch!='losing']
bowlers_df = bowlers_df[bowlers_df.WonByHowMuch!='default']

# Batters
batters_df = batters_df[batters_df.WonByHowMuch.isna()==False]
batters_df = batters_df[batters_df.WonByHowMuch!='losing']
batters_df = batters_df[batters_df.WonByHowMuch!='default']

In [16]:
# TO DO - split dismissals in the batters_df

### The cleaned and expanded dataframes

In [17]:
batters_df.T

0      \
Player                                       Mudassar Nazar   
Dismissal                                       b  De Silva   
Runs                                                     27   
BallsFaced                                               57   
Fours                                                     1   
Sixes                                                     0   
StrikeRate                                            47.37   
Date                                    1982-03-29 00:00:00   
Location                            Gaddafi Stadium, Lahore   
Result            Sri Lanka won by 30 runs [Revised Target]   
Innings                                                   1   
Team                                               Pakistan   
BatSideRR                                              5.98   
BatSideWicksLost                                          4   
BatSideScore                                            239   
Win                                                       0   
Year                                                   1982   
Month                                                     3   
Week                                                     13   
Day                                                      29   
Dayofweek                                                 0   
Dayofyear                                                88   
Is_month_end                                          False   
Is_month_start                                        False   
Is_quarter_end                                        False   
Is_quarter_start                                      False   
Is_year_end                                           False   
Is_year_start                                         False   
WinningTeam                                       Sri Lanka   
WonByHowMuch                                             30   
WinByHowType                                           runs   

                                                      1      \
Player                                          Mohsin Khan   
Dismissal                                           run out   
Runs                                                      6   
BallsFaced                                               14   
Fours                                                     0   
Sixes                                                     0   
StrikeRate                                            42.86   
Date                                    1982-03-29 00:00:00   
Location                            Gaddafi Stadium, Lahore   
Result            Sri Lanka won by 30 runs [Revised Target]   
Innings                                                   1   
Team                                               Pakistan   
BatSideRR                                              5.98   
BatSideWicksLost                                          4   
BatSideScore                                            239   
Win                                                       0   
Year                                                   1982   
Month                                                     3   
Week                                                     13   
Day                                                      29   
Dayofweek                                                 0   
Dayofyear                                                88   
Is_month_end                                          False   
Is_month_start                                        False   
Is_quarter_end                                        False   
Is_quarter_start                                      False   
Is_year_end                                           False   
Is_year_start                                         False   
WinningTeam                                       Sri Lanka   
WonByHowMuch                                             30   
WinByHowType                                           runs   

                    

In [18]:
bowlers_df.T

0      \
Player                                           A L F De Mel   
Overs                                                       8   
Maidens                                                     1   
Runs                                                       31   
Wickets                                                     0   
EconRate                                                 3.88   
Date                                      1982-03-29 00:00:00   
Location                              Gaddafi Stadium, Lahore   
Result              Sri Lanka won by 30 runs [Revised Target]   
Innings                                                     1   
Team                                                Sri Lanka   
BatSideRR                                                5.98   
BatSideWicketsLost                                          4   
BatSideScore                                              239   
Win                                                         1   
Year                                                     1982   
Month                                                       3   
Week                                                       13   
Day                                                        29   
Dayofweek                                                   0   
Dayofyear                                                  88   
Is_month_end                                            False   
Is_month_start                                          False   
Is_quarter_end                                          False   
Is_quarter_start                                        False   
Is_year_end                                             False   
Is_year_start                                           False   
WinningTeam                                         Sri Lanka   
WonByHowMuch                                               30   
WinByHowType                                             runs   

                                                        1      \
Player                                          J R Ratnayeke   
Overs                                                       8   
Maidens                                                     1   
Runs                                                       42   
Wickets                                                     1   
EconRate                                                 5.25   
Date                                      1982-03-29 00:00:00   
Location                              Gaddafi Stadium, Lahore   
Result              Sri Lanka won by 30 runs [Revised Target]   
Innings                                                     1   
Team                                                Sri Lanka   
BatSideRR                                                5.98   
BatSideWicketsLost                                          4   
BatSideScore                                              239   
Win                                                         1   
Year                                                     1982   
Month                                                       3   
Week                                                       13   
Day                                                        29   
Dayofweek                                                   0   
Dayofyear                                                  88   
Is_month_end                                            False   
Is_month_start                                          False   
Is_quarter_end                                          False   
Is_quarter_start                                        False   
Is_year_end                                             False   
Is_year_start                                           False   
WinningTeam                                         Sri Lanka   
WonByHowMuch                                               30   
WinByHowType                                             runs   

                        

In [19]:
bowlers_df.columns

Index(['Player', 'Overs', 'Maidens', 'Runs', 'Wickets', 'EconRate', 'Date',
       'Location', 'Result', 'Innings', 'Team', 'BatSideRR',
       'BatSideWicketsLost', 'BatSideScore', 'Win', 'Year', 'Month', 'Week',
       'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start',
       'WinningTeam', 'WonByHowMuch', 'WinByHowType'],
      dtype='object')

In [20]:
batters_df.columns

Index(['Player', 'Dismissal', 'Runs', 'BallsFaced', 'Fours', 'Sixes',
       'StrikeRate', 'Date', 'Location', 'Result', 'Innings', 'Team',
       'BatSideRR', 'BatSideWicksLost', 'BatSideScore', 'Win', 'Year', 'Month',
       'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end',
       'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end',
       'Is_year_start', 'WinningTeam', 'WonByHowMuch', 'WinByHowType'],
      dtype='object')

### Designating categorical and continuous variables

#### Batters

In [21]:
# Batter categorical and continuous variables TO DO
cat_vars = ['Player', 'Dismissal', 'Runs', 'BallsFaced', 'Fours', 'Sixes',
       'StrikeRate', 'Date', 'Location', 'Result', 'Innings', 'Team',
       'BatSideRR', 'BatSideWicksLost', 'BatSideScore', 'Win', 'WinningTeam',
       'WonByHowMuch', 'WinByHowType', 'Year', 'Month', 'Week', 'Day',
       'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start',
       'Elapsed']

cont_vars = []

#### Bowlers

In [24]:
# Bowler categorical and continuous variables
cat_vars_bowl = ['Player', 'Date', 'Location', 'Result', 'Innings', 'Team',  'Win', 
            'WinningTeam', 'WinByHowType', 'Year', 'Month', 'Week', 'Day', 
            'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']

cont_vars_bowl = ['Overs', 'Maidens', 'Runs', 'Wickets', 'EconRate',
            'BatSideRR', 'BatSideWicketsLost', 'BatSideScore', 'WonByHowMuch']

In [25]:
# Designating categorical variables in the bowlers_df
for cat_var in cat_vars_bowl:
    bowlers_df[cat_var] = bowlers_df[cat_var].astype('category').cat.as_ordered()

In [26]:
# Designating continuous variables in the bowlers_df
for cont_var in cont_vars_bowl:
    bowlers_df[cont_var] = bowlers_df[cont_var].astype('float32')

### Creating Embedding sizes

In [27]:
# Checking the size of each category
# The +1 is for an 'unknown' or 'other' column in the embedding matrix
cat_sz_bowl = [(column, len(bowlers_df[column].cat.categories)+1) for column in cat_vars_bowl]
cat_sz_bowl

[('Player', 1598),
 ('Date', 2966),
 ('Location', 191),
 ('Result', 1325),
 ('Innings', 3),
 ('Team', 27),
 ('Win', 4),
 ('WinningTeam', 24),
 ('WinByHowType', 8),
 ('Year', 38),
 ('Month', 13),
 ('Week', 54),
 ('Day', 32),
 ('Dayofweek', 8),
 ('Dayofyear', 366),
 ('Is_month_end', 3),
 ('Is_month_start', 3),
 ('Is_quarter_end', 3),
 ('Is_quarter_start', 3),
 ('Is_year_end', 3),
 ('Is_year_start', 3)]

In [29]:
# Creating the embedding sizes
emb_szs = [(cardinality, min(50, (cardinality+1)//2)) for _, cardinality in cat_sz_bowl]
emb_szs

[(1598, 50),
 (2966, 50),
 (191, 50),
 (1325, 50),
 (3, 2),
 (27, 14),
 (4, 2),
 (24, 12),
 (8, 4),
 (38, 19),
 (13, 7),
 (54, 27),
 (32, 16),
 (8, 4),
 (366, 50),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2)]

## Which bowlers are similar based on match statistics?

In [28]:
# The number of bowlers in the dataset
len(set(bowlers_df['Player']))

1597

In [30]:
# Subsetting for only the key columns
bowler_similarity_df = bowlers_df[['Player', 'Overs', 'Maidens', 'Runs', 'Wickets', 
                                  'EconRate', 'Location', 'BatSideWicketsLost', 
                                  'BatSideScore', 'WinningTeam', 
                                  'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear']]

In [48]:
X = bowler_similarity_df.groupby(by='Player').mean()

In [49]:
X

Overs   Maidens       Runs   Wickets   EconRate  \
Player                                                                 
A A Donald        8.767097  0.619355  36.490322  1.645161   4.175097   
A B Agarkar       8.459890  0.543956  42.895603  1.543956   5.174506   
A B Dinda         7.523077  0.153846  47.076923  0.923077   6.242308   
A B de Villiers   3.750000  0.000000  23.875000  0.875000   6.480000   
A Balbirnie       4.500000  0.000000  28.500000  1.000000   6.475000   
A Bhandari        8.700000  0.000000  53.000000  2.500000   5.770000   
A C Agar          7.600000  0.000000  45.400002  1.000000   5.956000   
A C Botha         8.000000  0.684211  35.789474  1.421053   4.305789   
A C Cummins       8.468056  0.388889  39.166668  1.222222   4.675972   
A C Dale          8.827586  1.137931  32.379311  1.068966   3.858621   
A C Dawson        8.172222  0.555556  39.555557  1.166667   4.901111   
A C Evans         8.853333  0.400000  44.266666  1.866667   5.026667   
A C Hudson        1.000000  0.000000   3.000000  0.000000   3.000000   
A C I Lock        6.871428  0.571429  31.285715  1.142857   4.578571   
A C Voges         3.583333  0.083333  20.166666  0.333333   6.045833   
A Codrington      6.250000  0.500000  33.125000  1.250000   6.657500   
A D Brown         1.000000  0.000000   5.000000  0.000000   5.000000   
A D Gaekwad       8.000000  0.000000  39.000000  1.000000   4.880000   
A D J Pitcher     4.366667  0.000000  26.666666  0.000000   5.776667   
A D Mascarenhas   7.444445  0.333333  33.888889  0.722222   5.671111   
A D Mathews       5.592126  0.346457  26.377953  0.748031   4.982047   
A D Mullally      9.244680  0.957447  35.680851  1.319149   3.903404   
A D R Campbell    2.816667  0.100000  14.466666  0.400000   5.864666   
A D Russell       7.346154  0.256410  43.589745  1.333333   5.895385   
A D S Fletcher    1.466667  0.000000   8.666667  0.000000   6.443333   
A Dananjaya       8.346666  0.000000  45.533333  1.866667   5.690000   
A F Giles         8.365385  0.307692  35.961540  1.019231   4.404231   
A F Milne         7.957576  0.212121  42.393940  1.000000   5.399394   
A Flintoff        7.854386  0.561404  35.096493  1.421053   4.510614   
A Flower          1.000000  0.000000   9.000000  0.000000   9.000000   
...                    ...       ...        ...       ...        ...   
Waqas Barkat      2.000000  0.000000  19.000000  0.000000   9.500000   
Wasim Akram       8.614118  0.673529  33.720589  1.426471   3.899118   
Wasim Haider      9.000000  0.500000  39.000000  0.500000   4.425000   
Wasim Raja        6.700000  0.307692  29.153847  0.538462   5.240769   
X J Doherty       8.428000  0.280000  40.720001  1.020000   4.995200   
X M Marshall      0.300000  0.000000   5.000000  0.000000  10.000000   
Y Goonasekera     3.000000  0.000000  17.500000  0.500000   5.835000   
Y K Pathan        4.995918  0.061224  27.428572  0.673469   5.893265   
Y Punja           6.000000  0.000000  49.000000  0.000000   8.170000   
Y S Chahal        9.073684  0.526316  39.210526  2.000000   4.300526   
Yamin Ahmadzai    6.000000  0.666667  30.000000  0.666667   4.166667   
Yashpal Sharma    4.328571  0.000000  26.714285  0.142857   6.300000   
Yasir Arafat      6.900000  0.200000  37.299999  0.400000   5.584000   
Yasir Hameed      1.000000  0.000000   8.666667  0.000000   8.666667   
Yasir Shah        8.831250  0.062500  47.875000  1.125000   5.553750   
Younis Khan       2.326316  0.052632  14.631579  0.105263   6.523158   
Yuvraj Singh      5.253247  0.116883  26.629869  0.701299   5.517013   
Z Khan            8.511170  0.601064  42.042553  1.425532   4.974308   
Zafar Gohar      10.000000  0.000000  54.000000  2.000000   5.400000   
Zafar Iqbal       4.714286  0.285714  19.571428  0.428571   5.554286   
Zaheer Abbas      3.866667  0.166667  18.583334  0.583333   4.935000   
Zaheer Maqsood   10.000000  0.000000  56.000000  2.000000   5.600000   
Zahid Ahmed       8.000000  0.500000  30.500000  1.500000   3

In [125]:
kmeans = KMeans(n_clusters=15, n_init=100).fit(X)

In [126]:
cluster_map = pd.DataFrame()
cluster_map['data_index'] = X.index.values
cluster_map['cluster'] = kmeans.labels_

In [127]:
cluster_map.groupby(by='cluster').count().plot.barh()

In [162]:
bowlers = set(cluster_map.data_index)

In [163]:
players_of_interest = ['G D McGrath', 'A A Donald', 'Waqar Younis', 'Wasim Akram', 'S K Warne',
              'M Muralitharan', 'R J Hadlee', 'D L Vettori', 'A Kumble', 'S C G MacGill',
              'D W Steyn', 'M J Clarke', 'M A Starc', 'D K Lillee']

In [164]:
cluster_map[cluster_map.data_index.isin(players_of_interest)].sort_values(by='cluster')

data_index  cluster
59          A Kumble        2
362       D K Lillee        2
371      D L Vettori        2
901   M Muralitharan        2
1281   S C G MacGill        2
1316       S K Warne        2
412        D W Steyn        5
833        M A Starc        5
868       M J Clarke        8
0         A A Donald       11
477      G D McGrath       11
1176      R J Hadlee       11
1566    Waqar Younis       11
1568     Wasim Akram       11